# asprin-vPL Tutorial

***

## 1. Introduction

*asprin-VPL* builds on the existing answer set programming (ASP) framework *asprin* and is designed to learn preferences using  labeled data. This tutorial assumes foundational knowledge in ASP syntax and ASP solver/grounder *clingo*. Prior experience with *asprin* would be helpful towards understanding this framework. Tutorial on preference optimization with *asprin* can be found in the *Potassco* user guide, under the chapter 'optimization and preference handling': https://github.com/potassco/guide/releases

Input:
1. pairs of models alongisde a label indicating the preferred model between the pair. The models are made up of binary atoms (either a fact or not).
2. the domain of the binary atoms and the total number of models for training.
3. the types of preference relation allowed and the corresponding range of preference instances as part of a preference statement, in the form of choice rules. 

Output:
1. preference statement(s) in the syntax of *asprin* which best fits the input data 

***

## 2. Example


In [7]:
%%file example_complete.lp

#program domain.
domain(2).                  % 1
atom(food(X)) :- domain(X). % 2
model(1..2).                % 3

#program examples.
in(food(1),1).              % 4
in(food(1),2).              % 5
in(food(2),2).              % 6
input(1,better,2).          % 7

#program generation.
preference(p1,subset).                                           % 8
{preference(p1,(1,X),1,for(atom(food(X))),())} :- atom(food(X)). % 9

Overwriting example_complete.lp


In [2]:
! clingo example_complete.lp backend.lp cleaned_asprin_lib.lp

'clingo' is not recognized as an internal or external command,
operable program or batch file.


## 3. Usage

### 3.1 Input
The input consists of 3 necessary program parts and 1 optional program part, each identified with its respective `#program <part>.` directive.

***

#### 3.1.1 Domain
This program part is identified by the directive `#program domain.`

```
#program domain.
domain(2).                  % 1
atom(food(X)) :- domain(X). % 2
model(1..2).                % 3
```

Line 1 specifys the domain space, which is the range of input allowed for the atom `food` in line 2. Line 3 states the total number of models given to the program.

#### 3.1.2 Example
This program part is identified by the directive `#program example.`

```
#program examples.
in(food(1),1).              % 4
in(food(1),2).              % 5
in(food(2),2).              % 6
input(1,better,2).          % 7
```

Lines 4 to 6 are the input predicates detailing the constitution of each model. The first argument is the atom and the second argument is the model in which the atom is present. In line 4, `in(food(1),1).` means the atom `food(1)` is a fact in model number `1`. Line 7 is the label stating that model `1` is preferred to model `2`. Other preference relations include 'worse', 'eq' for equal, and 'unc' for uncomparable.

#### 3.1.3 Generation
This program part is identified by the directive `#program generation.`

```
#program generation.
preference(p1,subset).                                           % 8
{preference(p1,(1,X),1,for(atom(food(X))),())} :- atom(food(X)). % 9
```

Line 8 specifies a preference statement p1 and its preference type, in this case subset. Line 9 is a choice rule stating the range of preference instances that preference statement p1 can possibly include. The syntax of the predicates `preference/2` and `preference/5` can be found in the tutorial for *asprin* mentioned in the introduction above.

### 3.2 Output
The output is the learned preference statement in asprin format.